<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c36fa979fb81f7c4ab237c4fecb7381201f06a7986941f01dc29b663fdc9c850
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 10:19:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -6.83 K (-0.05%)
Current PnL: -20.00 L (-13.45%)
CY Booked + Current PnL: -8.56 L (-5.75%)
-------------------
Total profit:  1.47 L
Total loss:  -21.47 L
-------------------
Total Booked + Current PnL: 18.42 L (15.14%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.2%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.86 L (61.51%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.31,59.0,H-MC,3.86,195500.0,30830.0,12492.0,0.56,18.72,6.39,26.31,89.0,2.47,1.40,28.20,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.44,42.0,M-SC,2.89,85071.0,-15706.0,15798.0,0.31,-15.58,18.57,0.09,245.0,-0.99,0.61,11.77,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,60.0,H-MC,10.55,111189.0,10685.0,18424.0,1.80,10.63,16.57,28.96,79.0,0.58,0.80,48.21,MH,ATH,METALS
84,VOLTAS,1530.00,-1.92,46.0,H-MC,3.05,205665.0,13923.0,23837.0,-0.09,7.26,11.59,19.69,99.0,0.58,1.47,14.55,XY25,NTT,AC
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214828.0,4462.0,24319.0,-0.43,2.12,11.32,13.68,37.0,0.18,1.54,18.99,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,10.55,111189.0,10685.0,18424.0,1.80,10.63,16.57,28.96,79.0,0.58,0.80,48.21,MH,ATH,METALS
32,HINDZINC,730.22,22.76,60.0,M-LC,2.43,200404.0,-4672.0,117236.0,1.72,-2.28,58.50,54.89,52.0,-0.04,1.44,21.27,X5K,ATH,METALS
66,SIS,528.00,2095.85,43.0,H-SC,4.45,88625.0,-22407.0,46014.0,1.71,-20.18,51.92,21.26,156.0,-0.49,0.64,19.74,OX40N,NTT,MISC
75,TITAGARH,1548.00,-2.76,54.0,H-SC,2.71,207947.0,-34230.0,145001.0,1.65,-14.13,69.73,45.74,158.0,-0.24,1.49,35.25,XY24,NTT,ENGINEERING
46,KPIGREEN,731.05,-0.19,39.0,H-SC,9.18,119020.0,-6277.0,65211.0,1.53,-5.01,54.79,47.03,141.0,-0.10,0.85,49.02,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-23.48,38.0,M-LC,2.29,242103.0,-34420.0,144487.0,-2.34,-12.45,59.68,39.81,54.0,-0.24,1.73,16.03,XY24,NTT,AUTO
71,TANLA,1963.11,-16.22,66.0,H-SC,16.18,224528.0,-46233.0,376174.0,-2.02,-17.08,167.54,121.86,133.0,-0.12,1.61,75.64,AR,ATH,IT
55,RAJOOENG,143.10,-41.05,27.0,H-SC,13.45,89240.0,-13260.0,53856.0,-1.66,-12.94,60.35,39.61,114.0,-0.25,0.64,0.00,AR,ATH,MISC
5,ASIANPAINT,3465.66,-15.68,35.0,H-LC,7.40,212705.0,-39063.0,92271.0,-1.63,-15.52,43.38,21.13,27.0,-0.42,1.52,14.41,X40,ATH,PAINTS
56,RELAXO,1176.00,-39.97,34.0,H-SC,2.28,81883.0,-63277.0,139201.0,-1.43,-43.59,170.00,52.31,136.0,-0.45,0.59,9.61,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.68,41.0,H-MC,2.13,134662.0,-1126.0,43873.0,-0.13,-0.83,32.58,31.48,107.0,-0.03,0.96,11.19,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,41.0,M-LC,5.48,151590.0,645.0,108811.0,-0.04,0.43,71.78,72.51,53.0,0.01,1.09,6.48,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.92,121500.0,-20979.0,40496.0,-0.02,-14.72,33.33,13.70,163.0,-0.52,0.87,43.40,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.44,42.0,M-SC,2.89,85071.0,-15706.0,15798.0,0.31,-15.58,18.57,0.09,245.0,-0.99,0.61,11.77,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,1.28,221103.0,-48564.0,84904.0,0.16,-18.01,38.40,13.47,138.0,-0.57,1.58,11.83,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,34.0,H-SC,1.99,140967.0,-34936.0,76954.0,0.11,-19.86,54.59,23.89,149.0,-0.45,1.01,21.74,OX40N,NTT,CERAMICS
66,SIS,528.0,2095.85,43.0,H-SC,4.45,88625.0,-22407.0,46014.0,1.71,-20.18,51.92,21.26,156.0,-0.49,0.64,19.74,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,50.76,70.0,M-MC,10.58,237550.0,12588.0,155073.0,1.18,5.60,65.28,74.53,192.0,0.08,1.70,39.65,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.31,59.0,H-MC,3.86,195500.0,30830.0,12492.0,0.56,18.72,6.39,26.31,89.0,2.47,1.40,28.20,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,3.05,205665.0,13923.0,23837.0,-0.09,7.26,11.59,19.69,99.0,0.58,1.47,14.55,XY25,NTT,AC
17,CAMS,5211.76,-3.61,51.0,H-SC,1.81,109600.0,7596.0,36332.0,0.80,7.45,33.15,43.06,122.0,0.21,0.79,25.84,X40N,ATH,MISC
23,DABUR,735.00,-6.72,44.0,H-MC,6.91,252540.0,4472.0,103945.0,0.56,1.80,41.16,43.70,102.0,0.04,1.81,16.51,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,10.55,111189.0,10685.0,18424.0,1.80,10.63,16.57,28.96,79.0,0.58,0.80,48.21,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-42.81,39.0,M-SC,1.60,96615.0,5116.0,73630.0,-0.60,5.59,76.21,86.07,223.0,0.07,0.69,40.30,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,66.0,H-MC,7.90,185806.0,5998.0,106411.0,-0.15,3.34,57.27,62.52,88.0,0.06,1.33,38.56,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,25.0,H-SC,10.45,125340.0,2004.0,131757.0,-0.50,1.63,105.12,108.45,119.0,0.02,0.90,24.34,AR,ATH,MISC
86,WIPRO,420.00,-13.79,41.0,M-LC,5.48,151590.0,645.0,108811.0,-0.04,0.43,71.78,72.51,53.0,0.01,1.09,6.48,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.59,34.0,H-LC,7.97,289843.0,-25999.0,138661.0,0.38,-8.23,47.84,35.67,5.0,-0.19,2.08,4.48,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-15.68,35.0,H-LC,7.40,212705.0,-39063.0,92271.0,-1.63,-15.52,43.38,21.13,27.0,-0.42,1.52,14.41,X40,ATH,PAINTS
74,TCS,4476.75,-29.01,38.0,H-LC,12.07,283410.0,-62546.0,137397.0,-0.67,-18.08,48.48,21.64,1.0,-0.46,2.03,0.40,X40,ATH,IT
42,ITC,452.00,-39.19,38.0,H-LC,2.08,195844.0,-4294.0,25186.0,-0.12,-2.15,12.86,10.44,4.0,-0.17,1.40,3.30,X40,NTT,FMCG
44,JIOFIN,387.00,-2.53,39.0,H-LC,13.15,264837.0,-2528.0,67984.0,0.39,-0.95,25.67,24.48,48.0,-0.04,1.90,53.51,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.78,45.0,H-LC,1.66,233730.0,-12421.0,91622.0,-0.25,-5.05,39.20,32.18,16.0,-0.14,1.67,29.18,X200,ATH,IT
42,ITC,452.0,-39.19,38.0,H-LC,2.08,195844.0,-4294.0,25186.0,-0.12,-2.15,12.86,10.44,4.0,-0.17,1.40,3.30,X40,NTT,FMCG
65,SIEMENS,4671.5,0.17,52.0,H-LC,2.83,160915.0,-25180.0,72653.0,0.54,-13.53,45.15,25.51,15.0,-0.35,1.15,18.58,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-10.26,48.0,H-LC,4.24,276307.0,10881.0,45922.0,0.19,4.10,16.62,21.40,11.0,0.24,1.98,11.02,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,43.0,H-LC,4.29,214828.0,4462.0,24319.0,-0.43,2.12,11.32,13.68,37.0,0.18,1.54,18.99,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.01,38.0,H-LC,12.07,283410.0,-62546.0,137397.0,-0.67,-18.08,48.48,21.64,1.0,-0.46,2.03,0.40,X40,ATH,IT
42,ITC,452.00,-39.19,38.0,H-LC,2.08,195844.0,-4294.0,25186.0,-0.12,-2.15,12.86,10.44,4.0,-0.17,1.40,3.30,X40,NTT,FMCG
83,VBL,671.64,-19.59,34.0,H-LC,7.97,289843.0,-25999.0,138661.0,0.38,-8.23,47.84,35.67,5.0,-0.19,2.08,4.48,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.51,41.0,H-LC,9.85,224981.0,-16943.0,80206.0,-0.64,-7.00,35.65,26.15,8.0,-0.21,1.61,6.32,X40,ATH,IT
1,ABB,7934.00,-40.06,47.0,H-LC,8.29,250594.0,-11025.0,130234.0,-0.32,-4.21,51.97,45.57,7.0,-0.08,1.80,6.63,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,52.0,H-LC,2.83,160915.0,-25180.0,72653.0,0.54,-13.53,45.15,25.51,15.0,-0.35,1.15,18.58,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,38.0,H-LC,2.08,195844.0,-4294.0,25186.0,-0.12,-2.15,12.86,10.44,4.0,-0.17,1.40,3.30,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,47.0,H-LC,4.97,211377.0,6877.0,34539.0,0.32,3.36,16.34,20.25,10.0,0.20,1.51,12.39,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-15.68,35.0,H-LC,7.40,212705.0,-39063.0,92271.0,-1.63,-15.52,43.38,21.13,27.0,-0.42,1.52,14.41,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214828.0,4462.0,24319.0,-0.43,2.12,11.32,13.68,37.0,0.18,1.54,18.99,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.01,38.0,H-LC,12.07,283410.0,-62546.0,137397.0,-0.67,-18.08,48.48,21.64,1.0,-0.46,2.03,0.40,X40,ATH,IT
40,INFY,2275.00,-18.16,48.0,H-LC,8.11,318350.0,5462.0,166211.0,-0.19,1.75,52.21,54.87,3.0,0.03,2.28,8.51,X40,BTT,IT
42,ITC,452.00,-39.19,38.0,H-LC,2.08,195844.0,-4294.0,25186.0,-0.12,-2.15,12.86,10.44,4.0,-0.17,1.40,3.30,X40,NTT,FMCG
83,VBL,671.64,-19.59,34.0,H-LC,7.97,289843.0,-25999.0,138661.0,0.38,-8.23,47.84,35.67,5.0,-0.19,2.08,4.48,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,47.0,H-LC,8.29,250594.0,-11025.0,130234.0,-0.32,-4.21,51.97,45.57,7.0,-0.08,1.80,6.63,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6772.22,51.0,L-SC,5.40,75399.0,-18411.0,95033.0,-0.80,-19.63,126.04,81.67,269.0,-0.19,0.54,46.01,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,43.01,84672.0,-28877.0,68931.0,-1.14,-25.43,81.41,35.27,268.0,-0.42,0.61,107.24,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.36,42.0,H-SC,11.45,87114.0,-13849.0,108379.0,0.11,-13.72,124.41,93.63,148.0,-0.13,0.62,29.75,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.69,94140.0,-3840.0,25446.0,-0.74,-3.92,27.03,22.05,152.0,-0.15,0.67,36.84,XR,ATH,FINANCE
50,NATIONALUM,244.55,-44.49,60.0,H-MC,10.55,111189.0,10685.0,18424.0,1.80,10.63,16.57,28.96,79.0,0.58,0.80,48.21,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1206.22,70.0,M-SC,11.47,177699.0,-3444.0,48921.0,-0.83,-1.90,27.53,25.10,235.0,-0.07,1.27,42.41,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-16.22,66.0,H-SC,16.18,224528.0,-46233.0,376174.0,-2.02,-17.08,167.54,121.86,133.0,-0.12,1.61,75.64,AR,ATH,IT
7,ATULAUTO,844.00,3727.03,54.0,M-SC,6.63,119251.0,-28117.0,79934.0,-0.47,-19.08,67.03,35.16,236.0,-0.35,0.85,21.80,XY24,NTT,AUTO
60,SAIL,228.00,50.76,70.0,M-MC,10.58,237550.0,12588.0,155073.0,1.18,5.60,65.28,74.53,192.0,0.08,1.70,39.65,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,59.23,58.0,H-SC,5.94,137825.0,-44950.0,156018.0,-0.38,-24.59,113.20,60.77,125.0,-0.29,0.99,23.44,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.21
1,25,43.67
2,50,74.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.41
MC    23.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.24
X40      14.73
X40N     11.45
XR       11.25
XY25     10.10
AR        8.34
OX40N     7.86
X200      1.67
MH        1.65
X5K       1.44
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.37
H-LC    25.97
H-MC    20.66
M-SC    13.03
M-LC     5.40
M-MC     2.68
L-SC     1.51
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.98,-5.23,39.96
IT,12.95,-16.23,79.61
FINANCE,10.05,-16.99,68.04
MISC,7.00,-17.86,83.39
BANKS,6.15,-16.15,77.25
PAINTS,5.62,-17.59,35.12
ELECTRICAL,5.49,-7.67,45.44
INSURANCE,3.71,-6.57,46.41
AC,3.57,2.82,13.45


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3158504.0,22
XR,1338839.0,14
AR,1166676.0,9
X40,791945.0,10
X40N,608452.0,9
OX40N,550382.0,10
XY25,429430.0,7
SR,249519.0,2
X5K,117236.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3694643.0,29
M-SC,1425944.0,17
H-MC,1222652.0,15
H-LC,1192839.0,15
M-LC,396988.0,4
M-MC,303953.0,2
L-SC,250340.0,3
L-MC,60317.0,1
L-LC,38564.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237818.0      6
M-SC       XY24       814171.0      7
H-SC       AR         795676.0      5
           XR         779567.0      7
H-MC       XY24       563637.0      4
H-LC       X40        501271.0      5
H-SC       X40N       307577.0      4
M-MC       XY24       303953.0      2
H-SC       OX40N      259275.0      4
           SR         249519.0      2
H-LC       X40N       222276.0      3
H-MC       X40        216798.0      4
H-LC       AR         202887.0      2
M-LC       XY24       170941.0      2
H-MC       XY25       169453.0      2
M-SC       AR         168113.0      2
L-SC       XR         163964.0      2
M-SC       OX40N      135401.0      4
           XR         119769.0      2
M-LC       X5K        117236.0      1
M-SC       XY25       114614.0      1
M-LC       XR         108811.0      1
H-LC       XY25       106799.0      3
H-MC       XR         106411.0      1
H-LC       X200        91622.0      1
L-SC       OX40N       86376.0      1
H-MC       X40N        78599.0      2
M-SC       X40         73876.0      1
H-MC       OX40N       69330.0      1
H-LC       XY24        67984.0      1
H-SC       MH          65211.0      1
L-MC       XR          60317.0      1
L-LC       XY25        38564.0      1
H-MC       MH          18424.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
